<a href="https://colab.research.google.com/github/IndiaFW/IndiaFW/blob/main/visionaries_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating Semantic Dementia
The goal of our project is to better understand the modelling approach and tools of Neuromatch Academy through simulation. 

We will simulate Semantic Dementia, a neurocognitive disorder that is characterised by loss of verbal, semantic discriminability. In our study, we reduce the dimensionality of information available for decoding and encoding models and examine how this loss of information affects classification accuracy. This is an extremely simplified illustration of semantic dementia but provides a foundation to use the tools we are learning.

We use the Kay natural image dataset for this simulation.



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.io import loadmat

#@title Preliminaries
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

def plot_stim_and_spikes(stim, spikes, dt, nt=120):
  """Show time series of stim intensity and spike counts.

  Args:
    stim (1D array): vector of stimulus intensities
    spikes (1D array): vector of spike counts
    dt (number): duration of each time step
    nt (number): number of time steps to plot

  """
  timepoints = np.arange(120)
  time = timepoints * dt

  f, (ax_stim, ax_spikes) = plt.subplots(
    nrows=2, sharex=True, figsize=(8, 5),
  )
  ax_stim.plot(time, stim[timepoints])
  ax_stim.set_ylabel('Stimulus intensity')

  ax_spikes.plot(time, spikes[timepoints])
  ax_spikes.set_xlabel('Time (s)')
  ax_spikes.set_ylabel('Number of spikes')

  f.tight_layout()


def plot_glm_matrices(X, y, nt=50):
  """Show X and Y as heatmaps.

  Args:
    X (2D array): Design matrix.
    y (1D or 2D array): Target vector.

  """
  from matplotlib.colors import BoundaryNorm
  from mpl_toolkits.axes_grid1 import make_axes_locatable
  Y = np.c_[y]  # Ensure Y is 2D and skinny

  f, (ax_x, ax_y) = plt.subplots(
    ncols=2,
    figsize=(6, 8),
    sharey=True,
    gridspec_kw=dict(width_ratios=(5, 1)),
  )
  norm = BoundaryNorm([-1, -.2, .2, 1], 256)
  imx = ax_x.pcolormesh(X[:nt], cmap="coolwarm", norm=norm)

  ax_x.set(
    title="X\n(lagged stimulus)",
    xlabel="Time lag (time bins)",
    xticks=[4, 14, 24],
    xticklabels=['-20', '-10', '0'],
    ylabel="Time point (time bins)",
  )
  plt.setp(ax_x.spines.values(), visible=True)

  divx = make_axes_locatable(ax_x)
  caxx = divx.append_axes("right", size="5%", pad=0.1)
  cbarx = f.colorbar(imx, cax=caxx)
  cbarx.set_ticks([-.6, 0, .6])
  cbarx.set_ticklabels(np.sort(np.unique(X)))

  norm = BoundaryNorm(np.arange(y.max() + 1), 256)
  imy = ax_y.pcolormesh(Y[:nt], cmap="magma", norm=norm)
  ax_y.set(
    title="Y\n(spike count)",
    xticks=[]
  )
  ax_y.invert_yaxis()
  plt.setp(ax_y.spines.values(), visible=True)

  divy = make_axes_locatable(ax_y)
  caxy = divy.append_axes("right", size="30%", pad=0.1)
  cbary = f.colorbar(imy, cax=caxy)
  cbary.set_ticks(np.arange(y.max()) + .5)
  cbary.set_ticklabels(np.arange(y.max()))

def plot_spike_filter(theta, dt, **kws):
  """Plot estimated weights based on time lag model.

  Args:
    theta (1D array): Filter weights, not including DC term.
    dt (number): Duration of each time bin.
    kws: Pass additional keyword arguments to plot()

  """
  d = len(theta)
  t = np.arange(-d + 1, 1) * dt

  ax = plt.gca()
  ax.plot(t, theta, marker="o", **kws)
  ax.axhline(0, color=".2", linestyle="--", zorder=1)
  ax.set(
    xlabel="Time before spike (s)",
    ylabel="Filter weight",
  )

def compute_accuracy(X, y, model):
  """Compute accuracy of classifier predictions.
  Args:
    X (2D array): Data matrix
    y (1D array): Label vector
    model (sklearn estimator): Classifier with trained weights.
  Returns:
    accuracy (float): Proportion of correct predictions.
  """

  y_pred = model.predict(X)

  accuracy = (y == y_pred).mean()

  return accuracy

def plot_spikes_with_prediction(
    spikes, predicted_spikes, dt, nt=50, t0=120, **kws):
  """Plot actual and predicted spike counts.

  Args:
    spikes (1D array): Vector of actual spike counts
    predicted_spikes (1D array): Vector of predicted spike counts
    dt (number): Duration of each time bin.
    nt (number): Number of time bins to plot
    t0 (number): Index of first time bin to plot.
    kws: Pass additional keyword arguments to plot()

  """
  t = np.arange(t0, t0 + nt) * dt

  f, ax = plt.subplots()
  lines = ax.stem(t, spikes[:nt], use_line_collection=True)
  plt.setp(lines, color=".5")
  lines[-1].set_zorder(1)
  kws.setdefault("linewidth", 3)
  yhat, = ax.plot(t, predicted_spikes[:nt], **kws)
  ax.set(
      xlabel="Time (s)",
      ylabel="Spikes",
  )
  ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
  ax.legend([lines[0], yhat], ["Spikes", "Predicted"])

  plt.show()

def plot_accuracy(the_data):
  """Plot accuracy of cross validation.

  Args:
    the_data (1D array): vector of crossvalidation accuracies
    
  """
  f, ax = plt.subplots(figsize=(8, 3))
  ax.boxplot(the_data, vert=False, widths=.7)
  ax.scatter(the_data, np.ones(8))
  ax.set(
    xlabel="Accuracy",
    yticks=[],
    title=f"Cross-validation accuracy: {accuracies.mean():.2%}"
  )
  ax.spines["left"].set_visible(False)


In [ ]:
#@title Load data
fname = "kay_labels.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/r638s/download
fname = "kay_labels_val.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/yqb3e/download
fname = "kay_images.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/ymnjv/download

# Simple Linear Regression approach
Let's try to predict neural response from image data. Project TA recommended we start with the standard decoding approach (predict image identity from neural data) and then reverse things. 

Here's the plan of attack recommended by Andrew:

1. PCA on the voxels
1. Train LDA on the top principle components to predict the image labels
1. Evaulate model fit
1. Cut out some of the weights (simulating a "loss of information" in the clinical condition)
1. Attempt evaluation in reverse (predict voxel PCAs from image data)

In [ ]:
#@title Load PCA helper functions

def plot_KAY_sample(X):
  """
  Plots 9 images in the Kay natural image dataset.

  Args:
     X (numpy array of floats) : Data matrix each column corresponds to a
                                 different image

  Returns:
    Nothing.

  """

  fig, ax = plt.subplots()
  k = 0
  for k1 in range(3):
    for k2 in range(3):
      k = k + 1
      plt.imshow(np.reshape(X[k,:],[128,128]),
                 extent=[(k1 + 1) * 128, k1 * 128, (k2+1) * 128, k2 * 128])

  plt.xlim((3 * 128, 0))
  plt.ylim((3 * 128, 0))
  plt.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False)
  ax.set_xticks([])
  ax.set_yticks([])
  plt.show()

def change_of_basis(X, W):
  """
  Projects data onto a new basis.

  Args:
    X (numpy array of floats) : Data matrix each column corresponding to a
                                different random variable
    W (numpy array of floats) : new orthonormal basis columns correspond to
                                basis vectors

  Returns:
    (numpy array of floats)   : Data matrix expressed in new basis
  """

  Y = np.matmul(X, W)

  return Y


def get_sample_cov_matrix(X):
  """
  Returns the sample covariance matrix of data X.

  Args:
    X (numpy array of floats) : Data matrix each column corresponds to a
                                different random variable

  Returns:
    (numpy array of floats)   : Covariance matrix
"""

  X = X - np.mean(X, 0)
  cov_matrix = 1 / X.shape[0] * np.matmul(X.T, X)
  return cov_matrix


def sort_evals_descending(evals, evectors):
  """
  Sorts eigenvalues and eigenvectors in decreasing order. Also aligns first two
  eigenvectors to be in first two quadrants (if 2D).

  Args:
    evals (numpy array of floats)    :   Vector of eigenvalues
    evectors (numpy array of floats) :   Corresponding matrix of eigenvectors
                                         each column corresponds to a different
                                         eigenvalue

  Returns:
    (numpy array of floats)          : Vector of eigenvalues after sorting
    (numpy array of floats)          : Matrix of eigenvectors after sorting
  """

  index = np.flip(np.argsort(evals))
  evals = evals[index]
  evectors = evectors[:, index]
  if evals.shape[0] == 2:
    if np.arccos(np.matmul(evectors[:, 0],
                           1 / np.sqrt(2) * np.array([1, 1]))) > np.pi / 2:
      evectors[:, 0] = -evectors[:, 0]
    if np.arccos(np.matmul(evectors[:, 1],
                           1 / np.sqrt(2)*np.array([-1, 1]))) > np.pi / 2:
      evectors[:, 1] = -evectors[:, 1]

  return evals, evectors




def plot_eigenvalues(evals, limit=True):
  """
  Plots eigenvalues.

  Args:
     (numpy array of floats) : Vector of eigenvalues

  Returns:
    Nothing.

  """

  plt.figure()
  plt.plot(np.arange(1, len(evals) + 1), evals, 'o-k')
  plt.xlabel('Component')
  plt.ylabel('Eigenvalue')
  plt.title('Scree plot')
  if limit:
    plt.show()

def pca(X):
  """
  Performs PCA on multivariate data. Eigenvalues are sorted in decreasing order

  Args:
     X (numpy array of floats) :   Data matrix each column corresponds to a
                                   different random variable

  Returns:
    (numpy array of floats)    : Data projected onto the new basis
    (numpy array of floats)    : Vector of eigenvalues
    (numpy array of floats)    : Corresponding matrix of eigenvectors

  """

  X = X - np.mean(X, 0)
  cov_matrix = get_sample_cov_matrix(X)
  evals, evectors = np.linalg.eigh(cov_matrix)
  evals, evectors = sort_evals_descending(evals, evectors)
  score = change_of_basis(X, evectors)

  return score, evectors, evals

In [ ]:
#@title Run PCA on voxel or image data
#@markdown Note, the image PCA takes about 15mins to run via Colab. 
#@markdown The voxel PCA is considerably faster (only 3mins).

#@markdown This cell also generates a Scree Plot, illustrating the top 50 components.

# Load data into data object
with np.load(fname) as dobj:
    dat = dict(**dobj)

## Neural data?
X = dat["responses"]

## Stimuli?
# We need to reshape the images into vectors to accord with MINST analysis code
# X = dat["stimuli"]
# X = np.reshape(X,[1750,(128*128)])

# Visualize
# plot_KAY_sample(X)

# Perform PCA
score, evectors, evals = pca(X)

# Plot the eigenvalues
plot_eigenvalues(evals, limit=False)
plt.xlim([0, 50])  # limit x-axis up to 100 for zooming

In [ ]:
#@title image PCA plotting tools

def reconstruct_data(score, evectors, X_mean, K):
  """
  Reconstruct the data based on the top K components.
  Args:
    score (numpy array of floats)    : Score matrix
    evectors (numpy array of floats) : Matrix of eigenvectors
    X_mean (numpy array of floats)   : Vector corresponding to data mean
    K (scalar)                       : Number of components to include
  Returns:
    (numpy array of floats)          : Matrix of reconstructed data
  """

  # Reconstruct the data from the score and eigenvectors
  # Don't forget to add the mean!!
  X_reconstructed =  np.matmul(score[:, :K], evectors[:, :K].T) + X_mean

  return X_reconstructed

def plot_KAY_reconstruction(X, X_reconstructed_first,X_reconstructed_second):
  """
  Plots 9 images in the MNIST dataset side-by-side with the reconstructed
  images.

  Args:
    X (numpy array of floats)               : Data matrix each column
                                              corresponds to a different
                                              random variable
    X_reconstructed (numpy array of floats) : Data matrix each column
                                              corresponds to a different
                                              random variable

  Returns:
    Nothing.
  """

  plt.figure()
  ax = plt.subplot(131)
  k = 0
  for k1 in range(3):
    for k2 in range(3):
      k = k + 1
      plt.imshow(np.reshape(X[k, :], (128, 128)),
                 extent=[(k1 + 1) * 128, k1 * 128, (k2 + 1) * 128, k2 * 128])
  plt.xlim((3 * 128, 0))
  plt.ylim((3 * 128, 0))
  plt.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False)
  ax.set_xticks([])
  ax.set_yticks([])
  plt.title('Data')
  ax = plt.subplot(132)
  k = 0
  for k1 in range(3):
    for k2 in range(3):
      k = k + 1
      plt.imshow(np.reshape(np.real(X_reconstructed_first[k, :]), (128, 128)),
                 extent=[(k1 + 1) * 128, k1 * 128, (k2 + 1) * 128, k2 * 128])
  plt.xlim((3 * 128, 0))
  plt.ylim((3 * 128, 0))
  plt.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False)
  ax.set_xticks([])
  ax.set_yticks([])
  plt.title('1st component')
  plt.tight_layout()
  ax = plt.subplot(133)
  k = 0
  for k1 in range(3):
    for k2 in range(3):
      k = k + 1
      plt.imshow(np.reshape(np.real(X_reconstructed_second[k, :]), (128, 128)),
                 extent=[(k1 + 1) * 128, k1 * 128, (k2 + 1) * 128, k2 * 128])
  plt.xlim((3 * 128, 0))
  plt.ylim((3 * 128, 0))
  plt.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False)
  ax.set_xticks([])
  ax.set_yticks([])
  plt.title('15th component')
  plt.tight_layout()

In [ ]:
K = 1

# Reconstruct the data based on all components
X_mean = np.mean(X, 0)
X_reconstructed = reconstruct_data(score, evectors, X_mean, K)

K = 15

# Reconstruct the data based on all components
X_mean = np.mean(X, 0)
X_reconstructed_2 = reconstruct_data(score, evectors, X_mean, K)

# Plot the data and reconstruction
plot_KAY_reconstruction(X, X_reconstructed,X_reconstructed_2)


Now that we have the principles components we can start to build a model.

In [ ]:


# LDA

# From labels to voxel space, expands dimensionality so much you'll run into problems



# Train on voxels to predict the images

# PCA on the voxels
# Train LDA on first X principles components to predict the labels
# Cut out some of the weights
# Try evaluation in reverse


#Logistic Regression approach
The next set of cells build a logistic regression that predicts the identity of an image (`animal` or `not animal`) based upon the neural data. 

In [ ]:
#@title Create training and test vectors
#@markdown This builds vectors of binarised data categorised as `animal` or `not animal`.
#@markdown We print a few cells to ensure it has worked.

# Load image labels for training images
labels = np.load('kay_labels.npy')

# Check labels at highest level of abstraction
find_animals = labels[0,:]
print(np.unique(find_animals))
print(find_animals[0:7])

# Find labels and build binarised list of animals and not animals
just_animals = np.array(range(len(find_animals)))
for i in range(len(find_animals)):
  if find_animals[i] == 'animal':
    just_animals[i] = 1
  else:
    just_animals[i] = 0

print(just_animals[0:7]) # OK

# Load image labels for validation images
val_labels = np.load('kay_labels_val.npy')

# Check labels at highest level of abstraction
find_animals = val_labels[0,:]
print(find_animals[0:7])

# Find labels and build binarised list of animals and not animals
test_animals = np.array(range(len(find_animals)))
for i in range(len(find_animals)):
  if find_animals[i] == 'animal':
    test_animals[i] = 1
  else:
    test_animals[i] = 0

print(test_animals[0:7]) # OK


We have successfully converted the training and test data into a training and test vector of binarised values where ratings of 1 are `animal` and ratings of 0 are `not animal`. 

Next, we pass the training vector to `sklearn.LogisticRegression` to create our model that attempts to predict the animal label from voxel activity.

In [ ]:
# Let's get some insight in the Kay data set with a view to decode it
with np.load(fname) as dobj:
    dat = dict(**dobj)

#print(dat["stimuli"].shape) # (1750,128,128) energy per pixel
#print(dat["responses"].shape) # (1750,8428) voxel activity per image 

# Can we predict stimulus as a function of voxel activity
# y = dat["stimuli"]
y = just_animals 
X = dat["responses"]

# # We need to reduce dimensionality of stimuli
# # This is a toy example, reducing to binarised activity for a single pixel
# y = y[:,64,64]

# # Convert to binary data (if statements don't work for whatever reason)
# mask = y > 0
# y[mask,] = 1
# mask = y <= 0
# y[mask,] = 0

# plt.hist(y,bins = 300)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Includes L2 "ridge" regularisation to reduce overfitting 
# L1 "sparseness" regularisation makes more sense but it took too long to run  
log_reg = LogisticRegression(penalty="l2", C=1, max_iter = 2000).fit(X,y)

# X_test = dat["responses_test"]
# y_pred = log_reg.predict(X)


We have now created the model (`log_reg`). To test the model we can crossvalidate and/or examine its accuracy using the test data.

In [ ]:
# Let's test accuracy using the test dataset

X_test = dat["responses_test"] # Here's the neural response during test
y_test = test_animals

# # We need to subset and binarise the stimuli data like we did for the training set
# y_test = y_test[:,64,64]

# # Convert to binary data (if statements don't work for whatever reason)
# mask = y_test > 0
# y_test[mask,] = 1
# mask = y_test <= 0
# y_test[mask,] = 0

# Compute train accuracy
test_accuracy = compute_accuracy(X_test, y_test, log_reg)
print(f"Accuracy on the test data: {test_accuracy:.2%}")

# Test accuracy is 61.67%
# Cross validation accuracy is 70.52%

accuracies = cross_val_score(log_reg, X, y, cv=8) # k=8 crossvalidation

plot_accuracy(accuracies)

Accuracy is OK although the cross-validation indicates we are overfitting (we have many more features than samples). We need to complete some form of dimension reduction on the neural data.

In [ ]:
# Time to do some PCA


## Neural Net approach
An alternative method to understand the Kay dataset is using a convolutional neural network to predict neural activity from images. 

This is a rough approximation of human visual perception so by reducing the type of information available in the image we may be able to simulate the type of symptoms that characterise semantic dementia.

Look at the approach by ...author... recommended by Philipp.

In [ ]:
# Here be CNNs

In [ ]:
#@title Playing with data
this = np.unique(labels[2,:])
print(this.shape)
# print(this)

that = np.unique(labels[3,:])
print(that.shape)
# print(that)

# dict(zip(np.unique(labels[0,:]), np.bincount(labels[0,:])))

# from collections import Counter

# this = Counter(labels[0,:])

# print(type(labels))

# this = np.unique(labels[0,:], return_counts=True)

# fig, ax = plt.subplots()    
# fig.set_size_inches(9, 5)
# ax.barh(y = this[0], width = this[1])
# plt.title("Labels at Level #1")
# plt.xlabel("Counts")
# for i, v in enumerate(this[1]):
#     ax.text(v+5, i-0.1, str(v), color='black')

# this = np.unique(labels[1,:], return_counts=True)

# fig, ax = plt.subplots() 
# fig.set_size_inches(9, 12)   
# ax.barh(y = this[0], width = this[1])
# plt.title("Labels at Level #2")
# plt.xlabel("Counts")
# for i, v in enumerate(this[1]):
#     ax.text(v+5, i-0.1, str(v), color='black')


this = labels[3,labels[2,:]=="mammal"]

this = np.unique(this, return_counts=True)

fig, ax = plt.subplots() 
fig.set_size_inches(9, 12)   
ax.barh(y = this[0], width = this[1])
plt.title("Categories within vertebrate animals")
plt.xlabel("Counts")
for i, v in enumerate(this[1]):
    ax.text(v+5, i-0.1, str(v), color='black')



In [ ]:
#@title LDA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
